In [62]:
import sys
sys.path.append('..')
from api.api import PDTB
from api.syntax_tree import Syntax_tree
from api.constituent import Constituent
from api.clause import Arg_Clauses
from api import util
import copy

In [5]:
# pdtb2 = PDTB('/home/pengfei/data/pdtb_v2/all/conll/')
pdtb3 = PDTB('/home/pengfei/data/pdtb3/all/conll/')

## previous sentence; Explicit

In [91]:
def _ps_arg1_clauses(parse_dict, Arg, DocID, sent_index, sent_length, sent_tokens, conn_token_indices):
    # first, use punctuation symbols to split the sentence
    punctuation = "...,:;?!~--"
    _clause_indices_list = []#[[(1,"I")..], ..]
    temp = []
    for index, word in sent_tokens:
        if word not in punctuation:
            temp.append((index, word))
        else:
            if temp != []:
                _clause_indices_list.append(temp)
                temp = []
    clause_indices_list = []
    for clause_indices in _clause_indices_list:
        temp = util.list_strip_punctuation(clause_indices)
        if temp != []:
            clause_indices_list.append([item[0] for item in temp])

    # then use SBAR tag in its parse tree to split each part into clauses.
    parse_tree = parse_dict[DocID]["sentences"][sent_index]["parsetree"].strip()
    syntax_tree = Syntax_tree(parse_tree)

    if syntax_tree.tree == None:
        return []

    clause_list = []
    for clause_indices in clause_indices_list:
        clause_tree = _get_subtree(syntax_tree, clause_indices)
        # 层次遍历，
        flag = 0
        for node in clause_tree.tree.traverse(strategy="levelorder"):
            if node.name == "SBAR":
                temp1 = [node.index for node in node.get_leaves()]
                temp2 = sorted(list(set(clause_indices) - set(temp1)))

                if temp2 == []:
                    clause_list.append(temp1)
                else:
                    if temp1[0] < temp2 [0]:
                        clause_list.append(temp1)
                        clause_list.append(temp2)
                    else:
                        clause_list.append(temp2)
                        clause_list.append(temp1)


                flag = 1
                break
        if flag == 0:
            clause_list.append(clause_indices)

    clauses = []# [([1,2,3],yes), ([4, 5],no), ]
    for clause_indices in clause_list:
        clauses.append((clause_indices, ""))

    gc = Arg_Clauses(None, Arg, DocID, sent_index, clauses)
    gc.conn_indices = conn_token_indices
    gc.conn_head_name = None
    return gc


In [101]:
def _ps_arg2_clauses(parse_dict, Arg, DocID, sent_index, sent_length, sent_tokens, conn_token_indices):
    # first, split the sentence by the connective and the punctuation symbols
    punctuation = "...,:;?!~--"
    _clause_indices_list = []#[[(1,"I")..], ..]
    temp = []
    for index, word in sent_tokens:
        if word not in punctuation and index not in conn_token_indices:
            temp.append((index, word))
        else:
            if temp != []:
                _clause_indices_list.append(temp)
                temp = []
    clause_indices_list = []
    for clause_indices in _clause_indices_list:
        temp = util.list_strip_punctuation(clause_indices)
        if temp != []:
            clause_indices_list.append([item[0] for item in temp])

    # then use SBAR tag in its parse tree to split each part into clauses.
    parse_tree = parse_dict[DocID]["sentences"][sent_index]["parsetree"].strip()
    syntax_tree = Syntax_tree(parse_tree)

    if syntax_tree.tree == None:
        return []

    clause_list = []
    for clause_indices in clause_indices_list:
        clause_tree = _get_subtree(syntax_tree, clause_indices)
        # BFS
        flag = 0
        for node in clause_tree.tree.traverse(strategy="levelorder"):
            if node.name == "SBAR":
                temp1 = [node.index for node in node.get_leaves()]
                temp2 = sorted(list(set(clause_indices) - set(temp1)))

                if temp2 == []:
                    clause_list.append(temp1)
                else:
                    if temp1[0] < temp2 [0]:
                        clause_list.append(temp1)
                        clause_list.append(temp2)
                    else:
                        clause_list.append(temp2)
                        clause_list.append(temp1)

                flag = 1
                break
        if flag == 0:
            clause_list.append(clause_indices)

    # print " ".join([parse_dict[DocID]["sentences"][sent_index]["words"][index][0] for index in range(sent_length)])
    # print clause_list
    # print Arg_list

    clauses = []# [([1,2,3],yes), ([4, 5],no), ]
    for clause_indices in clause_list:
        clauses.append((clause_indices, ""))

    gc = Arg_Clauses(1, Arg, DocID, sent_index, clauses)
    gc.conn_indices = conn_token_indices
    gc.conn_head_name = 'ignored'
    return gc

def _get_subtree(syntax_tree, clause_indices):
    copy_tree = copy.deepcopy(syntax_tree)

    for index, leaf in enumerate(copy_tree.tree.get_leaves()):
        leaf.add_feature("index",index)

    clause_nodes = []
    for index in clause_indices:
        node = copy_tree.get_leaf_node_by_token_index(index)
        clause_nodes.append(node)

    for node in copy_tree.tree.traverse(strategy="levelorder"):
        node_leaves = node.get_leaves()
        if set(node_leaves) & set(clause_nodes) == set([]):
            node.detach()
    return copy_tree

def check_if_just_contained(potential_constituents, arg):
    temp = []
    arg_set = set(arg)
    for c in potential_constituents:
        c_set = set(c)
        if len(c_set-arg_set)!=len(c_set):
            temp += arg_set
    if set(arg_set) != set(temp):
        return False
    else:
        return True

In [118]:
for i in range(len(pdtb3.parse_data)):
    if pdtb3.get_arg_sent_id(i, 'Arg1')[0] == pdtb3.get_arg_sent_id(i, 'Arg2')[0]-1 and pdtb3.parse_data[i]['Type'] == 'Explicit':
        try:
            DocID = pdtb3.parse_data[i]['DocID']
            conn_token_indices = [o[4] for o in pdtb3.parse_data[i]['Connective']['TokenList']]

            arg1 = 'Arg1'
            # assume arg1 and arg2 have only one sentence
            arg1_sent_index = pdtb3.get_arg_sent_id(i, 'Arg1')[0]
            sent_length = len(pdtb3.parse_dict[DocID]["sentences"][arg1_sent_index]["words"])
            sent_tokens = [(index, pdtb3.parse_dict[DocID]["sentences"][arg1_sent_index]["words"][index][0]) for index in range(0, sent_length)]
            gc = _ps_arg1_clauses(pdtb3.parse_dict, arg1, DocID, arg1_sent_index, sent_length, sent_tokens, conn_token_indices)
            potential_constituents = [o[0] for o in gc.clauses]
            arg = [o[1] for o in pdtb3.get_arg_token_list(i, arg1)]
            if not check_if_just_contained(potential_constituents, arg):
                pdtb3.get_highlighted_parsetree(i)


            arg2 = 'Arg2'
            # assume arg1 and arg2 have only one sentence
            arg2_sent_index = pdtb3.get_arg_sent_id(i, 'Arg2')[0]
            sent_length = len(pdtb3.parse_dict[DocID]["sentences"][arg2_sent_index]["words"])
            sent_tokens = [(index, pdtb3.parse_dict[DocID]["sentences"][arg2_sent_index]["words"][index][0]) for index in range(0, sent_length)]
            gc = _ps_arg2_clauses(pdtb3.parse_dict, arg2, DocID, arg2_sent_index, sent_length, sent_tokens, conn_token_indices)
            potential_constituents = [o[0] for o in gc.clauses]
            arg = [o[1] for o in pdtb3.get_arg_token_list(i, arg2)]
            if not check_if_just_contained(potential_constituents, arg):
                pdtb3.get_highlighted_parsetree(i)
        except Exception as e:
            print(i, e)


51 'list' object has no attribute 'clauses'
1091 'list' object has no attribute 'clauses'
1092 'list' object has no attribute 'clauses'
1334 'list' object has no attribute 'clauses'
2016 'list' object has no attribute 'clauses'
2035 'list' object has no attribute 'clauses'
2649 'list' object has no attribute 'clauses'
3772 'list' object has no attribute 'clauses'
4064 'list' object has no attribute 'clauses'
4653 'list' object has no attribute 'clauses'
5183 'list' object has no attribute 'clauses'
5185 'list' object has no attribute 'clauses'
5508 'list' object has no attribute 'clauses'
6282 'list' object has no attribute 'clauses'
7403 'list' object has no attribute 'clauses'
7462 'list' object has no attribute 'clauses'
7463 'list' object has no attribute 'clauses'
7485 'list' object has no attribute 'clauses'
7577 'list' object has no attribute 'clauses'
7978 'list' object has no attribute 'clauses'
8128 'list' object has no attribute 'clauses'
8150 'list' object has no attribute 

24571 'list' object has no attribute 'clauses'
24982 'list' object has no attribute 'clauses'
25635 'list' object has no attribute 'clauses'
25641 'list' object has no attribute 'clauses'
27121 'list' object has no attribute 'clauses'
27161 'list' object has no attribute 'clauses'
27163 'list' object has no attribute 'clauses'
27209 'list' object has no attribute 'clauses'
27211 'list' object has no attribute 'clauses'
28060 'list' object has no attribute 'clauses'
28062 'list' object has no attribute 'clauses'
29996 'list' object has no attribute 'clauses'
30075 'list' object has no attribute 'clauses'
30473 'list' object has no attribute 'clauses'
30950 'list' object has no attribute 'clauses'
31087 'list' object has no attribute 'clauses'
31105 'list' object has no attribute 'clauses'
31125 'list' object has no attribute 'clauses'
31138 'list' object has no attribute 'clauses'
31238 'list' object has no attribute 'clauses'
31362 'list' object has no attribute 'clauses'
31427 'list' 

In [127]:
# all of the index above the tree is not available
pdtb3.get_highlighted_parsetree(2649)

Relation: 2649,  Green for Connective(if any), Red for Arg1 and Blue for Arg2. 
DocID: wsj_0553,	Type: Explicit,	Sense: Expansion.Level-of-detail.Arg2-as-detail,
Arg1_sent_id: 49	,	Arg2_sent_id: 50


         /IN/-In
      /PP
     |  |   /DT/-the
     |   \NP
     |      \NN/-end
     |
     |-*COMMA*-*COMMA*
   /S|
  |  |   /DT/-neither
  |  |-NP
  |  |   \NN/-policy
  |  |
  |  |   /VBD-was
  |   \VP
  |      \VP/VBN-followed
  |
  |-*COMMA*-*COMMA*
  |
  |-CC/-and
  |
  |       /RB/-instead
  |   /ADVP
  |  |   |   /IN/-of
-S|  |    \PP
  |  |      |      /VBG-learning
  |  |       \S /VP
  |-S|             \NP/NN/-anything
  |  |
  |  |-NP/PRP-we
  |  |
  |  |   /VBP-are
  |   \VP
  |     |   /VBN-left
  |      \VP
  |        |   /IN/-with
  |         \PP
  |           |   /DT/-a
  |            \NP
  |               \NN/-mystery
  |
   \. /-.


AttributeError: 'NoneType' object has no attribute 'get_leaves'

## same sentence; Explicit

In [ ]:
def _get_constituents(parse_dict, DocID, sent_index, conn_indices):
    parse_tree = parse_dict[DocID]["sentences"][sent_index]["parsetree"].strip()
    syntax_tree = Syntax_tree(parse_tree)
    if syntax_tree.tree == None:
        return []
    constituent_nodes = []
    if len(conn_indices) == 1:# like and or so...
        conn_node = syntax_tree.get_leaf_node_by_token_index(conn_indices[0]).up
    else:
        conn_node = syntax_tree.get_common_ancestor_by_token_indices(conn_indices)
        conn_leaves = set([syntax_tree.get_leaf_node_by_token_index(conn_index) for conn_index in conn_indices])
        children = conn_node.get_children()
        for child in children:
            leaves = set(child.get_leaves())
            if conn_leaves & leaves == set([]):
                constituent_nodes.append(child)

    curr = conn_node
    while not curr.is_root():
        constituent_nodes.extend(syntax_tree.get_siblings(curr))
        curr = curr.up

    # obtain the Constituent object according to the node.
    constituents = []
    for node in constituent_nodes:
        cons = Constituent(syntax_tree, node)
        constituents.append(cons)
    return constituents



In [47]:
def check_if_just_contained(potential_constituents, arg):
    temp = []
    arg_set = set(arg)
    for c in potential_constituents:
        c = c.indices
        c_set = set(c)
        if len(c_set-arg_set)!=len(c_set):
            temp += arg_set
    if set(arg_set) != set(temp):
        return False
    else:
        return True

count = 0
for i in range(len(pdtb3.parse_data)):
    if pdtb3.get_arg_sent_id(i, 'Arg1')[0] == pdtb3.get_arg_sent_id(i, 'Arg2')[0] and pdtb3.parse_data[i]['Type'] == 'Explicit':
        try:
            docid = pdtb3.parse_data[i]['DocID']
            sent_index = pdtb3.get_arg_sent_id(i, 'Arg1')[0]
            conn_indices = [o[4] for o in pdtb3.parse_data[i]['Connective']['TokenList']]
            constituents = _get_constituents(pdtb3.parse_dict, docid, sent_index, conn_indices)
            arg1 = [o[4] for o in pdtb3.parse_data[i]['Arg1']['TokenList']]
            arg2 = [o[4] for o in pdtb3.parse_data[i]['Arg2']['TokenList']]
            a = check_if_just_contained(constituents, arg1)
            b = check_if_just_contained(constituents, arg2)
            if not (a and b):
                count += 1
                pdtb3.get_highlighted_parsetree(i)
        except Exception as e:
            print(e)

Relation: 106,  Green for Connective(if any), Red for Arg1 and Blue for Arg2. 
DocID: wsj_0799,	Type: Explicit,	Sense: Expansion.Conjunction,
Arg1_sent_id: 46	,	Arg2_sent_id: 46


      /DT/-The
   /NP
  |   \NN/-company
  |
  |        /RB/-not
  |   /CONJP
  |  |     \RB/-only
  |  |
  |  |   /VBD-merged
  |  |  |
  |  |-VP   /CD/-three
  |  |  |  |
  |  |  |  |-JJ/-military-electronics
  |  |   \NP
  |  |     |-NN/-manufacturing
  |-VP     |
  |  |      \NNS-operations
  |  |
  |  |-*COMMA*-*COMMA*
  |  |
  |  |-CC/-but
  |  |
  |  |   /ADVPRB/-also
  |  |  |
  |  |  |-VBD-closed
  |  |  |
-S|   \VP      /DT/-an
  |     |     |
  |     |   /NP-JJ/-unrelated
  |     |  |  |
  |     |  |   \NN/-plant
  |      \NP
  |        |    /WHNPWDT-that
  |        |   |
  |         \SBAR     /VBZ-makes
  |            |     |
  |             \S /VP      /JJ/-ordnance
  |                  |   /NP
  |                  |  |   \NNS-devices
  |                   \NP
  |                     |   /VBN-use

Relation: 4814,  Green for Connective(if any), Red for Arg1 and Blue for Arg2. 
DocID: wsj_0629,	Type: Explicit,	Sense: Expansion.Conjunction,
Arg1_sent_id: 44	,	Arg2_sent_id: 44

'NoneType' object has no attribute 'get_leaves'
Relation: 4815,  Green for Connective(if any), Red for Arg1 and Blue for Arg2. 
DocID: wsj_0629,	Type: Explicit,	Sense: Expansion.Conjunction,
Arg1_sent_id: 44	,	Arg2_sent_id: 44

'NoneType' object has no attribute 'get_leaves'
Relation: 4967,  Green for Connective(if any), Red for Arg1 and Blue for Arg2. 
DocID: wsj_0694,	Type: Explicit,	Sense: Contingency.Cause.Reason,
Arg1_sent_id: 8	,	Arg2_sent_id: 8

'NoneType' object has no attribute 'get_leaves'
Relation: 4969,  Green for Connective(if any), Red for Arg1 and Blue for Arg2. 
DocID: wsj_0694,	Type: Explicit,	Sense: Contingency.Cause.Reason,
Arg1_sent_id: 8	,	Arg2_sent_id: 8

'NoneType' object has no attribute 'get_leaves'
Relation: 5163,  Green for Connective(if any), Red for Arg1 and Blue for Arg2. 
DocID:

Relation: 7671,  Green for Connective(if any), Red for Arg1 and Blue for Arg2. 
DocID: wsj_0277,	Type: Explicit,	Sense: Contingency.Cause.Reason,
Arg1_sent_id: 30	,	Arg2_sent_id: 30


      /NP/PRP-It
     |
     |   /VBZ-'s
   /S|  |
  |  |  |   /VBN-known
  |  |  |  |
  |   \VP  |   /IN/-for
  |     |  |-PP
  |     |  |  |   /PRP$-its
  |     |  |   \NP
  |     |  |      \NN/-cautiousness
  |      \VP
  |        |   /IN/-in
  |        |-PP
  |        |   \NP/NN/-part
  |        |
  |        |    /IN/-because
  |        |   |
  |         \SBAR  /NP/PRP-it
  |            |  |
  |             \S|   /VBZ-has
  |               |  |
  |                \VP   /VBN-had
  |                  |  |
  |                   \VP      /JJ/-little
  |                     |   /NP
  |                     |  |   \NN/-need
  |                      \NP
  |                        |   /IN/-for
  |                        |  |
  |                         \PP   /JJ/-bold
  |                           |  |
  |    

Relation: 11494,  Green for Connective(if any), Red for Arg1 and Blue for Arg2. 
DocID: wsj_1574,	Type: Explicit,	Sense: Expansion.Conjunction,
Arg1_sent_id: 18	,	Arg2_sent_id: 18

'NoneType' object has no attribute 'get_leaves'
Relation: 11495,  Green for Connective(if any), Red for Arg1 and Blue for Arg2. 
DocID: wsj_1574,	Type: Explicit,	Sense: Temporal.Asynchronous.Succession,
Arg1_sent_id: 18	,	Arg2_sent_id: 18

'NoneType' object has no attribute 'get_leaves'
Relation: 11498,  Green for Connective(if any), Red for Arg1 and Blue for Arg2. 
DocID: wsj_1574,	Type: Explicit,	Sense: Contingency.Condition.Arg2-as-cond,
Arg1_sent_id: 19	,	Arg2_sent_id: 19

'NoneType' object has no attribute 'get_leaves'
Relation: 11513,  Green for Connective(if any), Red for Arg1 and Blue for Arg2. 
DocID: wsj_1574,	Type: Explicit,	Sense: Contingency.Cause.Result,
Arg1_sent_id: 28	,	Arg2_sent_id: 28

'NoneType' object has no attribute 'get_leaves'
Relation: 11514,  Green for Connective(if any), Red for A

Relation: 14841,  Green for Connective(if any), Red for Arg1 and Blue for Arg2. 
DocID: wsj_0996,	Type: Explicit,	Sense: Temporal.Asynchronous.Precedence,
Arg1_sent_id: 1	,	Arg2_sent_id: 1

'NoneType' object has no attribute 'get_leaves'
Relation: 14843,  Green for Connective(if any), Red for Arg1 and Blue for Arg2. 
DocID: wsj_0996,	Type: Explicit,	Sense: Contingency.Condition.Arg2-as-cond,
Arg1_sent_id: 1	,	Arg2_sent_id: 1

'NoneType' object has no attribute 'get_leaves'
Relation: 14844,  Green for Connective(if any), Red for Arg1 and Blue for Arg2. 
DocID: wsj_0996,	Type: Explicit,	Sense: Comparison.Contrast,
Arg1_sent_id: 1	,	Arg2_sent_id: 1

'NoneType' object has no attribute 'get_leaves'
Relation: 14846,  Green for Connective(if any), Red for Arg1 and Blue for Arg2. 
DocID: wsj_0996,	Type: Explicit,	Sense: Expansion.Conjunction,
Arg1_sent_id: 1	,	Arg2_sent_id: 1

'NoneType' object has no attribute 'get_leaves'
Relation: 14854,  Green for Connective(if any), Red for Arg1 and Blue 

...

Relation: 23062,  Green for Connective(if any), Red for Arg1 and Blue for Arg2. 
DocID: wsj_1903,	Type: Explicit,	Sense: Expansion.Conjunction,
Arg1_sent_id: 13	,	Arg2_sent_id: 13

'NoneType' object has no attribute 'get_leaves'
Relation: 23085,  Green for Connective(if any), Red for Arg1 and Blue for Arg2. 
DocID: wsj_1903,	Type: Explicit,	Sense: Temporal.Asynchronous.Succession,
Arg1_sent_id: 32	,	Arg2_sent_id: 32


      /NNP-Mr.
   /NP
  |   \NNP-Lang
  |
  |   /VBD-surprised
  |  |
  |  |-NP/NNP-Time
  |  |
  |  |-ADVPRB/-soon
  |  |
  |  |   /IN/-after
  |  |  |
  |-VP  |         /VBG-joining
  |  |  |        |
  |  |  |        |-NP/NNS-forces
  |  |  |      /VP
  |  |  |     |  |    /WHADVPWRB-when
  |  |  |     |  |   |
  |  |  |     |   \SBAR  /NP/PRP-he
  |  |  |     |      |  |
  |  |  |     |       \S|   /VBD-said
  |   \PP     |         |  |
  |     |     |          \VP       /NP/PRP-he
  |     |     |            |      |
  |     |     |             \SBARS|   /MD/-would
  

Relation: 31274,  Green for Connective(if any), Red for Arg1 and Blue for Arg2. 
DocID: wsj_1120,	Type: Explicit,	Sense: Contingency.Condition.Arg2-as-cond,
Arg1_sent_id: 58	,	Arg2_sent_id: 58


       /IN/-If
      |
      |      /DT/-the
      |   /NP
      |  |   \NNP-LDP
   /SBAR |
  |   |  |-*COMMA*-*COMMA*
  |   |  |
  |   |  |    /IN/-as
  |   |  |-SBAR
  |    \S|   |   /ADVPRB/-currently
  |      |    \S|
  |      |       \VP/VBN-composed
  |      |
  |      |-*COMMA*-*COMMA*
  |      |
  |      |   /MD/-ca
  |      |  |
  |       \VP-RB/-n't
  |         |
  |         |   /VB/-make
  |          \VP
  |            |   /DT/-the
-S|             \NP
  |                \NN/-transformation
  |
  |-*COMMA*-*COMMA*
  |
  |-ADVPRB/-then
  |
  |-NP/PRP-it
  |
  |   /MD/-should
  |-VP
  |  |   /VB/-split
  |   \VP
  |     |   /IN/-into
  |      \PP
  |        |   /CD/-two
  |         \NP
  |            \NNS-parties
  |
   \. /-.



Relation: 31355,  Green for Connective(if any), Red for A

Relation: 35222,  Green for Connective(if any), Red for Arg1 and Blue for Arg2. 
DocID: wsj_1398,	Type: Explicit,	Sense: Contingency.Condition.Arg2-as-cond,
Arg1_sent_id: 3	,	Arg2_sent_id: 3

'NoneType' object has no attribute 'get_leaves'
Relation: 35291,  Green for Connective(if any), Red for Arg1 and Blue for Arg2. 
DocID: wsj_1302,	Type: Explicit,	Sense: Contingency.Cause.Reason,
Arg1_sent_id: 48	,	Arg2_sent_id: 48

'NoneType' object has no attribute 'get_leaves'
Relation: 35410,  Green for Connective(if any), Red for Arg1 and Blue for Arg2. 
DocID: wsj_1388,	Type: Explicit,	Sense: Temporal.Synchronous,
Arg1_sent_id: 11	,	Arg2_sent_id: 11

'NoneType' object has no attribute 'get_leaves'
Relation: 35411,  Green for Connective(if any), Red for Arg1 and Blue for Arg2. 
DocID: wsj_1388,	Type: Explicit,	Sense: Expansion.Manner.Arg2-as-manner,
Arg1_sent_id: 11	,	Arg2_sent_id: 11

'NoneType' object has no attribute 'get_leaves'
Relation: 35471,  Green for Connective(if any), Red for Arg1 

Relation: 38441,  Green for Connective(if any), Red for Arg1 and Blue for Arg2. 
DocID: wsj_1633,	Type: Explicit,	Sense: Contingency.Cause.Reason,
Arg1_sent_id: 2	,	Arg2_sent_id: 2

'NoneType' object has no attribute 'get_leaves'
Relation: 38596,  Green for Connective(if any), Red for Arg1 and Blue for Arg2. 
DocID: wsj_1671,	Type: Explicit,	Sense: Temporal.Asynchronous.Succession,
Arg1_sent_id: 1	,	Arg2_sent_id: 1


       /RB/-Now
      |
      |    /IN/-that
   /ADVP  |
  |   |   |      /DT/-the
  |   |   |     |
  |    \SBAR  /NP-NNP-Supreme
  |       |  |  |
  |       |  |   \NNP-Court
  |       |  |
  |        \S|   /VBD-opened
  |          |  |
  |          |  |   /DT/-the
  |          |  |-NP
  |           \VP   \NN/-door
  |             |
  |             |   /IN/-on
  |             |  |
  |              \PP      /DT/-the
  |                |   /NP
  |                |  |   \NN/-subject
  |                 \NP
  |                   |   /IN/-of
  |                    \PP
-S|    

Relation: 44016,  Green for Connective(if any), Red for Arg1 and Blue for Arg2. 
DocID: wsj_2276,	Type: Explicit,	Sense: Temporal.Asynchronous.Succession,
Arg1_sent_id: 88	,	Arg2_sent_id: 88


      /DT/-The
   /NP
  |   \NN/-market
  |
-S|   /VBD-closed
  |  |
  |  |-NP/NNS-minutes
   \VP
     |   /IN/-after
     |  |
     |  |      /VBG-reaching
      \PP     |
        |     |   /PRP$-its
        |     |-NP
         \S /VP   \NN/-high
              |
              |   /IN/-for
              |  |
               \PP      /DT/-the
                 |   /NP
                  \NP   \NN/-day
                    |
                     \PP/IN/-of



Relation: 44083,  Green for Connective(if any), Red for Arg1 and Blue for Arg2. 
DocID: wsj_2223,	Type: Explicit,	Sense: Expansion.Conjunction,
Arg1_sent_id: 4	,	Arg2_sent_id: 4

'NoneType' object has no attribute 'get_leaves'
Relation: 44108,  Green for Connective(if any), Red for Arg1 and Blue for Arg2. 
DocID: wsj_2223,	Type: Explicit,	Sense: E

In [48]:
count

546

## Implicit

In [137]:
def _arg_clauses(parse_dict, Arg, DocID, sent_index, sent_tokens):
#     DocID = relation["DocID"]
#     Arg_sent_indices = sorted([item[3] for item in relation[Arg]["TokenList"]])
#     Arg_token_indices = sorted([item[4] for item in relation[Arg]["TokenList"]])

#     if len(set(Arg_sent_indices)) != 1:
#         return []
#     relation_ID = relation["ID"]
#     sent_index = Arg_sent_indices[0]

#     sent_tokens = [(index, parse_dict[DocID]["sentences"][sent_index]["words"][index][0]) for index in Arg_token_indices]

    punctuation = "...,:;?!~--"
    # first, use punctuation symbols to split the sentence
    _clause_indices_list = []#[[(1,"I")..], ..]
    temp = []
    for index, word in sent_tokens:
        if word not in punctuation:
            temp.append((index, word))
        else:
            if temp != []:
                _clause_indices_list.append(temp)
                temp = []
    if temp != []:
        _clause_indices_list.append(temp)

    clause_indices_list = []
    for clause_indices in _clause_indices_list:
        temp = util.list_strip_punctuation(clause_indices)
        if temp != []:
            clause_indices_list.append([item[0] for item in temp])

    # then use SBAR tag in its parse tree to split each part into clauses.
    parse_tree = parse_dict[DocID]["sentences"][sent_index]["parsetree"].strip()
    syntax_tree = Syntax_tree(parse_tree)

    if syntax_tree.tree == None:
        return []

    clause_list = []
    for clause_indices in clause_indices_list:
        clause_tree = _get_subtree(syntax_tree, clause_indices)
        # BFS，
        flag = 0
        for node in clause_tree.tree.traverse(strategy="levelorder"):
            if node.name == "SBAR":
                temp1 = [node.index for node in node.get_leaves()]
                temp2 = sorted(list(set(clause_indices) - set(temp1)))

                if temp2 == []:
                    clause_list.append(temp1)
                else:
                    if temp1[0] < temp2 [0]:
                        clause_list.append(temp1)
                        clause_list.append(temp2)
                    else:
                        clause_list.append(temp2)
                        clause_list.append(temp1)
                flag = 1
                break
        if flag == 0:
            clause_list.append(clause_indices)
    clauses = []# [([1,2,3],yes), ([4, 5],no), ]
    for clause_indices in clause_list:
        clauses.append((clause_indices, ""))


    # print DocID, sent_index
    # print " ".join([parse_dict[DocID]["sentences"][sent_index]["words"][index][0] for index in Arg_token_indices])
    # print clauses

    return Arg_Clauses(None, Arg, DocID, sent_index, clauses)

In [145]:
err_count = 0
for i in range(len(pdtb3.parse_data)):
    if len(pdtb3.get_arg_sent_id(i, 'Arg1')) == len(pdtb3.get_arg_sent_id(i, 'Arg2')) == 1 and pdtb3.parse_data[i]['Type'] == 'Implicit':
        try:



            DocID = pdtb3.parse_data[i]['DocID']

            arg1 = 'Arg1'
            # assume arg1 and arg2 have only one sentence
            arg1_sent_index = pdtb3.get_arg_sent_id(i, 'Arg1')[0]
            sent_length = len(pdtb3.parse_dict[DocID]["sentences"][arg1_sent_index]["words"])
            sent_tokens = [(index, pdtb3.parse_dict[DocID]["sentences"][arg1_sent_index]["words"][index][0]) for index in range(0, sent_length)]
            arg_clauses = _arg_clauses(pdtb3.parse_dict, arg1, DocID, arg1_sent_index, sent_tokens)
            potential_constituents = [o[0] for o in arg_clauses.clauses]
            arg = [o[1] for o in pdtb3.get_arg_token_list(i, arg1)]
            if not check_if_just_contained(potential_constituents, arg):
                pdtb3.get_highlighted_parsetree(i)


            arg2 = 'Arg2'
            # assume arg1 and arg2 have only one sentence
            arg2_sent_index = pdtb3.get_arg_sent_id(i, 'Arg2')[0]
            sent_length = len(pdtb3.parse_dict[DocID]["sentences"][arg2_sent_index]["words"])
            sent_tokens = [(index, pdtb3.parse_dict[DocID]["sentences"][arg2_sent_index]["words"][index][0]) for index in range(0, sent_length)]
            arg_clauses = _arg_clauses(pdtb3.parse_dict, arg2, DocID, arg2_sent_index, sent_tokens)
            potential_constituents = [o[0] for o in arg_clauses.clauses]
            arg = [o[1] for o in pdtb3.get_arg_token_list(i, arg2)]
            if not check_if_just_contained(potential_constituents, arg):
                pdtb3.get_highlighted_parsetree(i)
            
            
            
            
            
            
            
        except Exception as e:
            print(i, e)
            err_count += 1

52 'list' object has no attribute 'clauses'
241 'list' object has no attribute 'clauses'
242 'list' object has no attribute 'clauses'
516 'list' object has no attribute 'clauses'
581 'list' object has no attribute 'clauses'
582 'list' object has no attribute 'clauses'
680 'list' object has no attribute 'clauses'
681 'list' object has no attribute 'clauses'
682 'list' object has no attribute 'clauses'
771 'list' object has no attribute 'clauses'
772 'list' object has no attribute 'clauses'
813 'list' object has no attribute 'clauses'
1089 'list' object has no attribute 'clauses'
1090 'list' object has no attribute 'clauses'
1126 'list' object has no attribute 'clauses'
1128 'list' object has no attribute 'clauses'
1130 'list' object has no attribute 'clauses'
1133 'list' object has no attribute 'clauses'
1253 'list' object has no attribute 'clauses'
1256 'list' object has no attribute 'clauses'
1341 'list' object has no attribute 'clauses'
1553 'list' object has no attribute 'clauses'
1

8302 'list' object has no attribute 'clauses'
8303 'list' object has no attribute 'clauses'
8304 'list' object has no attribute 'clauses'
8407 'list' object has no attribute 'clauses'
8421 'list' object has no attribute 'clauses'
8422 'list' object has no attribute 'clauses'
8467 'list' object has no attribute 'clauses'
8468 'list' object has no attribute 'clauses'
8479 'list' object has no attribute 'clauses'
8480 'list' object has no attribute 'clauses'
8481 'list' object has no attribute 'clauses'
8560 'list' object has no attribute 'clauses'
8561 'list' object has no attribute 'clauses'
8576 'list' object has no attribute 'clauses'
8636 'list' object has no attribute 'clauses'
8650 'list' object has no attribute 'clauses'
8681 'list' object has no attribute 'clauses'
8682 'list' object has no attribute 'clauses'
8718 'list' object has no attribute 'clauses'
8719 'list' object has no attribute 'clauses'
8752 'list' object has no attribute 'clauses'
8754 'list' object has no attribut

17146 'list' object has no attribute 'clauses'
17147 'list' object has no attribute 'clauses'
17180 'list' object has no attribute 'clauses'
17207 'list' object has no attribute 'clauses'
17208 'list' object has no attribute 'clauses'
17449 'list' object has no attribute 'clauses'
17450 'list' object has no attribute 'clauses'
17451 'list' object has no attribute 'clauses'
17512 'list' object has no attribute 'clauses'
17513 'list' object has no attribute 'clauses'
17514 'list' object has no attribute 'clauses'
17516 'list' object has no attribute 'clauses'
17574 'list' object has no attribute 'clauses'
17622 'list' object has no attribute 'clauses'
17623 'list' object has no attribute 'clauses'
17708 'list' object has no attribute 'clauses'
17738 'list' object has no attribute 'clauses'
17749 'list' object has no attribute 'clauses'
17750 'list' object has no attribute 'clauses'
17803 'list' object has no attribute 'clauses'
17804 'list' object has no attribute 'clauses'
17850 'list' 

25797 'list' object has no attribute 'clauses'
25798 'list' object has no attribute 'clauses'
25799 'list' object has no attribute 'clauses'
25834 'list' object has no attribute 'clauses'
25836 'list' object has no attribute 'clauses'
25882 'list' object has no attribute 'clauses'
26084 'list' object has no attribute 'clauses'
26086 'list' object has no attribute 'clauses'
26088 'list' object has no attribute 'clauses'
26119 'list' object has no attribute 'clauses'
26120 'list' object has no attribute 'clauses'
26121 'list' object has no attribute 'clauses'
26122 'list' object has no attribute 'clauses'
26247 'list' object has no attribute 'clauses'
26428 'list' object has no attribute 'clauses'
26429 'list' object has no attribute 'clauses'
26431 'list' object has no attribute 'clauses'
26781 'list' object has no attribute 'clauses'
26782 'list' object has no attribute 'clauses'
27000 'list' object has no attribute 'clauses'
27027 'list' object has no attribute 'clauses'
27123 'list' 

35091 'list' object has no attribute 'clauses'
35092 'list' object has no attribute 'clauses'
35220 'list' object has no attribute 'clauses'
35221 'list' object has no attribute 'clauses'
35292 'list' object has no attribute 'clauses'
35409 'list' object has no attribute 'clauses'
35509 'list' object has no attribute 'clauses'
35644 'list' object has no attribute 'clauses'
35697 'list' object has no attribute 'clauses'
35759 'list' object has no attribute 'clauses'
35774 'list' object has no attribute 'clauses'
35775 'list' object has no attribute 'clauses'
35848 'list' object has no attribute 'clauses'
35849 'list' object has no attribute 'clauses'
35939 'list' object has no attribute 'clauses'
36001 'list' object has no attribute 'clauses'
36009 'list' object has no attribute 'clauses'
36012 'list' object has no attribute 'clauses'
36062 'list' object has no attribute 'clauses'
36063 'list' object has no attribute 'clauses'
36081 'list' object has no attribute 'clauses'
36134 'list' 

44965 'list' object has no attribute 'clauses'
44966 'list' object has no attribute 'clauses'
44969 'list' object has no attribute 'clauses'
44971 'list' object has no attribute 'clauses'
44972 'list' object has no attribute 'clauses'
44973 'list' object has no attribute 'clauses'
44979 'list' object has no attribute 'clauses'
44980 'list' object has no attribute 'clauses'
44981 'list' object has no attribute 'clauses'
44998 'list' object has no attribute 'clauses'
44999 'list' object has no attribute 'clauses'
45001 'list' object has no attribute 'clauses'
45002 'list' object has no attribute 'clauses'
45083 'list' object has no attribute 'clauses'
45084 'list' object has no attribute 'clauses'
45104 'list' object has no attribute 'clauses'
45105 'list' object has no attribute 'clauses'
45106 'list' object has no attribute 'clauses'
45174 'list' object has no attribute 'clauses'
45258 'list' object has no attribute 'clauses'
45259 'list' object has no attribute 'clauses'
45354 'list' 

In [142]:
arg_clauses

[]